# Variables as a state in TensorFlow

In [1]:
import collections
import tensorflow as tf
import numpy as np

In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 1.10.0


## Variable creation and call

Initialize a variable. **[NOTE]** This cannot be called more than once.

In [3]:
with tf.variable_scope("test0") as scope:
    x = tf.get_variable(name="x", shape=[3], dtype=tf.float32, initializer=tf.random_normal_initializer)

Call the pre-defined variable with `reuse=True` option. 

In [4]:
with tf.variable_scope("test0", reuse=True) as scope:
    y = tf.get_variable(name="x", shape=[3], dtype=tf.float32)

The variable access is global hence you can even call from different scope.

In [5]:
def call_within_function_scope():
    with tf.variable_scope("test0", reuse=True) as scope:
        return tf.get_variable(name="x", shape=[3], dtype=tf.float32)

In [6]:
z = call_within_function_scope()

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_np, y_np, z_np = sess.run((x, y, z))
    print("Tensors' ids are the same?:", id(x) == id(y) == id(z))
    print("Numpy arrays ids are the same?", id(x_np) == id(y_np) == id(z_np))

Tensors' ids are the same?: True
Numpy arrays ids are the same? True


## Fibonacci sequence with variable assignment

Demonstrate variable assignment by computing Fibonacci sequence.

$$
\begin{pmatrix}
    F_{n+1} \\
    F_{n+2}
\end{pmatrix}
=
\begin{pmatrix}
    0 & 1 \\
    1 & 1 
\end{pmatrix}
\begin{pmatrix}
    F_{n} \\
    F_{n+1}
\end{pmatrix}
$$

In [8]:
## Again, you cannot run this cell more than once

with tf.variable_scope("var") as scope:
    variable = tf.get_variable(name="pair", dtype=tf.int32, initializer=[0, 1])

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    with tf.variable_scope("var", reuse=True) as scope:
        vv = tf.get_variable(name="pair", dtype=tf.int32)
    print(sess.run(vv))
    assign_op = tf.assign(vv, [vv[1], vv[0] + vv[1]])
    for _ in range(10):
        sess.run(assign_op)
        print(sess.run(vv))

[0 1]
[1 1]
[1 2]
[2 3]
[3 5]
[5 8]
[ 8 13]
[13 21]
[21 34]
[34 55]
[55 89]


## Input, output, and state

Previously we only had a state $S$. Now we incorporate input $X$ and output $Y$ together with state updates such that $(X, S) \to (Y, S)$. Specifically, we try to compute

$$
\begin{align*}
S_\textrm{new} &= \frac{X - S}{2} \\
Y &= X + S
\end{align*}
$$

In [10]:
## You cannot run this cell more than once

with tf.variable_scope("hello") as scope:
    s = tf.get_variable(name="s", dtype=tf.float32, shape=[2], trainable=False, initializer=tf.zeros_initializer)

In [11]:
def compute(inputs, scope_name="hello"):
    with tf.variable_scope(scope_name, reuse=True):
        state = tf.get_variable(name="s", trainable=False)

    outputs = inputs + state
    new_state = 0.5 * (inputs - state)
    return {"output": outputs, "state": new_state}

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    input_ = tf.placeholder(dtype=tf.float32, shape=[2])
    output = compute(input_)
    for _ in range(5):
        output_dict = sess.run((output), feed_dict={input_: [1.0, 2.0]})
        print(output_dict) 
        # assign the updated s value to the variable s.
        assign_op = tf.assign(s, output_dict["state"])
        sess.run(assign_op)

{'output': array([1., 2.], dtype=float32), 'state': array([0.5, 1. ], dtype=float32)}
{'output': array([1.5, 3. ], dtype=float32), 'state': array([0.25, 0.5 ], dtype=float32)}
{'output': array([1.25, 2.5 ], dtype=float32), 'state': array([0.375, 0.75 ], dtype=float32)}
{'output': array([1.375, 2.75 ], dtype=float32), 'state': array([0.3125, 0.625 ], dtype=float32)}
{'output': array([1.3125, 2.625 ], dtype=float32), 'state': array([0.34375, 0.6875 ], dtype=float32)}


## Compute and assign at once

Previous example was little clumsy in that update calculation and state-value assignment were done separately. To do them at once, we need to add the tensor-assigning operation to the computational graph. `tf.control_dependencies` does the job. **[NOTE]** There is subtlty in 

In [13]:
def compute_and_update_state(input_, scope_name="hello"):
    with tf.variable_scope(scope_name, reuse=True):
        state = tf.get_variable(name="s", trainable=False)
    output = input_ + state
    new_state = 0.5 * (input_ - state)
    
    ## [NOTE] this assign_op location matters!
    ##        the execution order is disturbed (in rabdom manner) 
    ##        if the op is stated outside of the context manager
    ##        Ref: https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/Graph#control_dependencies
    with tf.control_dependencies([output, new_state]):
        assign_op = tf.assign(state, new_state)  
        with tf.control_dependencies([assign_op]):
            output_ = tf.identity(output, name="out")
    return {"output": output_, "state": new_state}

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    inputs = tf.placeholder(dtype=tf.float32, shape=[2])
    output = compute_and_update_state(inputs)
    for _ in range(5):
        output_dict = sess.run((output), feed_dict={inputs: np.array([1., 2.])})
        print(output_dict)

{'output': array([1., 2.], dtype=float32), 'state': array([0.5, 1. ], dtype=float32)}
{'output': array([1.5, 3. ], dtype=float32), 'state': array([0.25, 0.5 ], dtype=float32)}
{'output': array([1.25, 2.5 ], dtype=float32), 'state': array([0.375, 0.75 ], dtype=float32)}
{'output': array([1.375, 2.75 ], dtype=float32), 'state': array([0.3125, 0.625 ], dtype=float32)}
{'output': array([1.3125, 2.625 ], dtype=float32), 'state': array([0.34375, 0.6875 ], dtype=float32)}
